## Configurando modelo

In [1]:
! pip install openai

In [2]:
API_KEY_CCC = "LL-8TEmrzbYgkmK420Q2gMNmvzoNL2JhY7rNyp62M0qzqyuUvwfvMTIHVQxJI6TeVn9"
API_KEY_LSD = 'LL-ZxxVaYWmfSsdYu9s3fonTJG4z53vWeulO9HTJlAH2haW2qMNfAIjzc5EkLepqdvv'

### Testing prompt

In [3]:
questão = """QUESTÃO 126  
Um garoto comprou vários abacates na feira, mas 
descobriu que eles não estavam maduros o suficiente 
para serem consumidos. Sua mãe recomendou que ele 
colocasse os abacates em um recipiente fechado, pois 
isso aceleraria seu amadurecimento. Com certa dúvida, o 
garoto realizou esta experiência: colocou alguns abacates 
no recipiente e deixou os demais em uma fruteira aberta. 
Surpreendendo-se, ele percebeu que os frutos que estavam 
no recipiente fechado amadureceram mais rapidamente.
 A aceleração desse processo é causada por"""
alternativas = """
A acúmulo de gás etileno.
B redução da umidade do ar.
C aumento da concentração de CO2.
D diminuição da intensidade luminosa.
E isolamento do contato com O2 atmosférico."""

In [4]:
# This example is the new way to use the OpenAI lib for python
from openai import OpenAI

client = OpenAI(
api_key = "LL-8TEmrzbYgkmK420Q2gMNmvzoNL2JhY7rNyp62M0qzqyuUvwfvMTIHVQxJI6TeVn9",
base_url = "https://api.llama-api.com"
)

conteudo = "Você deve responder perguntas em português sobre diversos temas. Suas respostas devem sem compostas com APENAS a letra da alternativa correta, não retorne mais nada para o user. Answer only with the character of the correct alternative. A,B,C,D or E."

response = client.chat.completions.create(
    model="llama-70b-chat",
    max_tokens= 30,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that should answer questions in Portuguese about various topics. Completing the phrase: A resposta correta é:. You must not answer more than the letter of the alternative"},
        {"role": "user", "content": f"com base nesse enunciado: {questão} \n responda qual a letra da alternativa correta, entre as opções a seguir: \n {alternativas} \n a alternativa correta é:"}
    ],
    
)

#print(response)
print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

{
  "id": null,
  "choices": [
    {
      "finish_reason": "max_token",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "A resposta correta é: A acúmulo de gás etileno. \n\nExplanation: \nO",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1709908271,
  "model": "llama-70b-chat",
  "object": null,
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 24,
    "prompt_tokens": 294,
    "total_tokens": 318
  }
}
A resposta correta é: A acúmulo de gás etileno. 

Explanation: 
O


### Experiment

In [3]:
from openai import OpenAI

context = "You are going to answer questions in Portuguese about various topics. Your answers must be composed with ONLY the letter of the correct alternative, do not return anything else to the user, such as explanation and text, only the letter. Completing the phrase:  A resposta correta é: "

def submit_questions(prompt):    
    
    client = OpenAI(
        api_key = API_KEY_LSD,
        base_url = "https://api.llama-api.com"
    )
    
    response = client.chat.completions.create(
        model="llama-70b-chat",
        max_tokens= 15,
        messages=[
            {"role": "system", "content": f'{context}'},
            {"role": "user", "content": f'{prompt}'}
        ],
        
    )
    # print(response.choices[0].message.content.replace('\n', ''))

    if response.choices[0].message.content == "" or response.choices[0].message.content == None or len(response.choices[0].message.content) < 3:
        raise Exception("A resposta está vazia")
    
    if ( not " A" in response.choices[0].message.content) and ( not "B" in response.choices[0].message.content) and ( not "C" in response.choices[0].message.content) and ( not "D" in response.choices[0].message.content) and ( not "E" in response.choices[0].message.content):
        raise Exception("A resposta não contém a letra da alternativa correta")
   
    return response.choices[0].message.content.replace('\n', '')

In [4]:
CSVS_PATH = '../TCC/csvs'
EXPERIMENTS_PATH = '../TCC/experiments'
EXPERIMENT_NUMBER = 1

import pandas as pd
import os

years = [2011,2012,2013]

for year in years:
    
    enem_csv = pd.read_csv(f'{CSVS_PATH}/enem_{year}_questions.csv').copy()
    model_answers = ''    
    problematic_questions = []
    problematic_outputs = []
    
    for index, row in enem_csv.iterrows():
        
        question = row['body']
        alternatives = row['alternatives']
        prompt = f'usando como base esse enunciado: {question} \n responda qual a alternativa correta, entre as opções a seguir: \n {alternatives} \n a alternativa correta é:'
        teste = ''
        
        try:
            
            teste = submit_questions(prompt)
            print(teste)
            
        except:
            problematic_questions.append(row['id'])
            
            with open(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/{year}/Llama2_{year}_answers.txt', 'w+',encoding='utf-8') as f:
                f.write(model_answers)
                
        model_answers += f'{row["id"]}\n{teste}\n'
        
    
    model_answers += f'\nQuestões problemáticas: {problematic_questions}'
    model_answers += f'\nSaídas problemáticas: {problematic_outputs}'
    
    with open(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/{year}/Llama2_{year}_answers.txt', 'w+',encoding='utf-8') as f:
        f.write(model_answers)

A resposta correta é E. Difundir ideias revol
A resposta correta é: B. Manifestou-se contra
A alternativa correta é D.Explanation:
A resposta correta é E.
A resposta correta é A industrialização voltada para o setor
A resposta correta é B, o terraceamento.
A resposta correta é: E reduzir o impacto ambient
A resposta correta é: CExplanation:
A resposta correta é: B. O rebaixamento do
A resposta correta é: E.
BExplanation:The correct answer is B, dimin
A resposta correta é D.
A resposta correta é D.
A resposta correta é: A - busca de oportun
A resposta correta é C. O avanço do trabalho
A resposta correta é C. Os movimentos sociais
A resposta correta é E) agrária, marcada pela concent
A resposta correta é B.
A resposta correta é C.
A resposta correta é: C 1960-1
A resposta correta é D.
A resposta correta é: B. A existência da lei não
A resposta correta é B. Fornecer informações que f
A resposta correta é A. Defina seus projetos a partir
A resposta correta é: C pela participação ativa das


### Resubmetendo as questões não respondidas

In [12]:
dic = {'questions' :
    [ """QUESTÃO 23  
 A estrela sobe
Marques Rebelo
O trabalho não era penoso: colar rótulos, meter 
vidros em caixas, etiquetá-las, selá-las, envolvê-las em 
papel celofane, branco, verde, azul, conforme o produto, 
separá-las em dúzias... Era fastidioso. Para passar mais 
rapidamente as oito horas havia o remédio: conversar. 
Era proibido, mas quem ia atrás de proibições? O patrão 
vinha? Vinha o encarregado do serviço? Calavam o 
bico, aplicavam-se ao trabalho. Mal viravam as costas, 
voltavam a taramelar. As mãos não paravam, as línguas 
não paravam. Nessas conversas intermináveis, de 
linguagem solta e assuntos crus, Leniza se completou. 
Isabela, Afonsina, Idália, Jurete, Deolinda – foram mestras. 
O mundo acabou de se desvendar. Leniza perdeu o tom 
ingênuo que ainda podia ter. Ganhou um jogar de corpo 
que convida, um quebrar de olhos que promete tudo, à 
toa, gratuitamente. Modificou-se o timbre de sua voz. 
Ficou mais quente. A própria inteligência se transformou. 
Tornou-se mais aguda, mais trepidante.
O romance, de 1939, traz à cena tipos e situações que 
espelham o Rio de Janeiro daquela década. No fragmento, 
o narrador delineia esse contexto centrado no""",
 """QUESTÃO 51  
Na obra Suma Teológica, Tomás de Aquino escreve:
Desde que tenhamos compreendido o significado 
da palavra “Deus”, sabemos, de imediato, que Deus 
existe. Com efeito, essa palavra designa uma coisa de 
tal ordem que não podemos conceber nada que lhe seja 
maior. Ora, o que existe na realidade e no pensamento é 
maior do que o que existe apenas no pensamento. Donde 
se segue que o objeto designado pela palavra “Deus”, 
que existe no pensamento, desde que se entenda essa 
palavra, também existe na realidade. Por conseguinte, a 
existência de Deus é evidente.
O texto apresenta uma elaboração teórica de Tomás de 
Aquino caracterizada por""",
"""QUESTÃO 131  
Em desenhos animados é comum vermos a 
personagem tentando impulsionar um barco soprando ar 
contra a vela para compensar a falta de vento. Algumas 
vezes usam o próprio fôlego, foles ou ventiladores. 
Estudantes de um laboratório didático resolveram 
investigar essa possibilidade. Para isso, usaram dois 
pequenos carros de plástico, A e B, instalaram sobre 
estes pequenas ventoinhas e fixaram verticalmente uma 
cartolina de curvatura parabólica para desempenhar 
uma função análoga à vela de um barco. No carro B 
inverteu-se o sentido da ventoinha e manteve-se a vela, 
a fim de manter as características físicas do barco, 
massa e formato da cartolina. A montagem do carro A 
busca simular a situação dos desenhos animados, pois 
a ventoinha está direcionada para a vela. A cartolina foi 
instalada à direita e a ventoinha foi instalada à esquerda, 
em ambos os carros.
Os estudantes ligaram as ventoinhas, aguardaram o 
fluxo de ar ficar permanente e determinaram os módulos 
das velocidades médias dos carros A (VA) e B (VB) para o 
mesmo intervalo de tempo.
a respeito das intensidades das velocidades médias e do sentido de movimento do carro A, os estudantes observaram que:""",
"""QUESTÃO 161  
O Salão do Automóvel de São Paulo é um evento 
no qual vários fabricantes expõem seus modelos mais 
recentes de veículos, mostrando, principalmente, suas 
inovações em design e tecnologia.
Uma montadora pretende participar desse evento 
com dois estandes, um na entrada e outro na região 
central do salão, expondo, em cada um deles, um carro 
compacto e uma caminhonete.
Para compor os estandes, foram disponibilizados 
pela montadora quatro carros compactos, de modelos 
distintos, e seis caminhonetes de diferentes cores para 
serem escolhidos aqueles que serão expostos. A posição 
dos carros dentro de cada estande é irrelevante.
Uma expressão que fornece a quantidade de maneiras 
diferentes que os estandes podem ser compostos é"""
 ],
'alternatives' : 
    ["""A julgamento da mulher fora do espaço doméstico.
B relato sobre as condições de trabalho no Estado Novo.
C destaque a grupos populares na condição de protagonistas.
D processo de inclusão do palavrão nos hábitos de linguagem.
E vínculo entre as transformações urbanas e os papéis femininos.""", 
"""A reiterar a ortodoxia religiosa contra os heréticos.
B sustentar racionalmente doutrina alicerçada na fé.
C explicar as virtudes teologais pela demonstração.
D flexibilizar a interpretação oficial dos textos sagrados.
E justificar pragmaticamente crença livre de dogmas.""",
"""A Velocidade do carro A é igual a zero; Velocidade do carro B é maior que zero; o carro A não se move.
B Zero é menor que a velocidade do carro A e a velocidade do carro A é menor que a velocidade do carro B; o carro A se move para a direita.
C Zero é menor que a velocidade do carro A e a velocidade do carro A é menor que a velocidade do carro B; o carro A se move para a esquerda.
D Zero é menor que a velocidade do carro B e a velocidade do carro B é menor que a velocidade do carro A; o carro A se move para a direita.
E Zero é menor que a velocidade do carro B e a velocidade do carro B é menor que a velocidade do carro A; o carro A se move para a esquerda.""",
""""""

]}

for i in range(len(dic['questions'])):
        
        question = dic['questions'][i]
        alternatives = dic['alternatives'][i]
        
        prompt = f'usando como base esse enunciado: {question} \n responda qual a alternativa correta, entre as opções a seguir: \n {alternatives}'
        
        try:
                answer = submit_questions(prompt)
                        
        except:
                print('Resending question')
                continue
        print(answer)

A resposta correta é: C - destaque a grupos populares
BExplanation:The text presents a rational elaboration
A resposta correta é: B Zero é menor que a velocidade
A resposta correta é: B combinação de 10
